Question 3

You are given a binary tree that represents cities where a certain corporation wishes to provide service by
constructing a service center. Building service centers at any node, i.e. a city can give service to its
immediate ancestor and descendant node (cities), returns the smallest number of service centers required
by the corporation to provide service to all connected cities

Input: tree= {0,0, null, 0, null, 0, null, null, 0}

Output: 2 

In [1]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def minServiceCenters(root):
    
    NOT_COVERED = 0
    COVERED_NO_CENTER = 1
    HAS_CENTER = 2

  
    total_centers = 0

    def dfs(node):
        nonlocal total_centers

     
        if node is None:
            return COVERED_NO_CENTER

        left_status = dfs(node.left)
        right_status = dfs(node.right)

       
        if left_status == NOT_COVERED or right_status == NOT_COVERED:
            total_centers += 1
            return HAS_CENTER

  
        if left_status == HAS_CENTER or right_status == HAS_CENTER:
            return COVERED_NO_CENTER

        return NOT_COVERED

    root_state = dfs(root)

    
    if root_state == NOT_COVERED:
        total_centers += 1

    return total_centers


In [3]:

root = TreeNode(0)
root.left = TreeNode(0)
root.left.right = TreeNode(0)
root.left.right.right = TreeNode(0)
root.left.right.right.right = TreeNode(0)

print(minServiceCenters(root)) 


2
